## Example: ingress data from OpenWeatherMap

This notebook will walk through an example of getting data from the OpenWeatherMap API, and inserting it into the DTBase database.

Prerequisites:
* DTBase backend running on `http://localhost:5000`
* Environment setup by sourcing `.secrets/dtenv.sh` , including the variable CONST_OPENWEATHERMAP_APIKEY which should point to an API key from https://openweathermap.org/

In [ ]:
import logging
logging.getLogger('matplotlib').setLevel(logging.WARNING)
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
from datetime import datetime, timedelta
from dtbase.ingress.ingress_weather import import_openweathermap_data
dt_to = datetime.now()
import_openweathermap_data(dt_to, "OpenWeatherMapHistory", create_sensors=True)
dt_to = datetime.now() + timedelta(days=2)
import_openweathermap_data(dt_to, "OpenWeatherMapForecast", create_sensors=False)

### Check that we now have the data in the database

In [ ]:
import requests
import pandas as pd

In [ ]:
r=requests.get("http://localhost:5000/sensor/list-sensors", json={"type_name": "weather"})
r.json()

Let's get temperature values for both the history "sensor" and the forecast "sensor"

In [ ]:
from datetime import datetime, timedelta
now = datetime.now().isoformat()
yesterday = (datetime.now() - timedelta(days=1)).isoformat()
tomorrow = (datetime.now() + timedelta(days=1)).isoformat()


In [ ]:
r = requests.get("http://localhost:5000/sensor/sensor-readings", json={"measure_name": "temperature", "unique_identifier": "OpenWeatherMapHistory", "dt_from": yesterday, "dt_to": now})
history_temps = r.json()
r = requests.get("http://localhost:5000/sensor/sensor-readings", json={"measure_name": "temperature", "unique_identifier": "OpenWeatherMapForecast", "dt_from": now, "dt_to": tomorrow})
forecast_temps = r.json()

In [ ]:
history_df = pd.DataFrame(history_temps)
forecast_df = pd.DataFrame(forecast_temps)

In [ ]:
history_df